In [139]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
from sklearn.metrics import roc_curve,auc

In [247]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [248]:
train.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [249]:
train.shape 


(200000, 202)

In [250]:
test.shape

(200000, 201)

In [251]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 202 entries, ID_code to var_199
dtypes: float64(200), int64(1), object(1)
memory usage: 308.2+ MB


In [48]:
train.describe()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,0.100490,10.679914,-1.627622,10.715192,6.796529,11.078333,-5.065317,5.408949,16.545850,0.284162,...,3.234440,7.438408,1.927839,3.331774,17.993784,-0.142088,2.303335,8.908158,15.870720,-3.326537
std,0.300653,3.040051,4.050044,2.640894,2.043319,1.623150,7.863267,0.866607,3.418076,3.332634,...,4.559922,3.023272,1.478423,3.992030,3.135162,1.429372,5.454369,0.921625,3.010945,10.438015
min,0.000000,0.408400,-15.043400,2.117100,-0.040200,5.074800,-32.562600,2.347300,5.349700,-10.505500,...,-14.093300,-2.691700,-3.814500,-11.783400,8.694400,-5.261000,-14.209600,5.960600,6.299300,-38.852800
25%,0.000000,8.453850,-4.740025,8.722475,5.254075,9.883175,-11.200350,4.767700,13.943800,-2.317800,...,-0.058825,5.157400,0.889775,0.584600,15.629800,-1.170700,-1.946925,8.252800,13.829700,-11.208475
50%,0.000000,10.524750,-1.608050,10.580000,6.825000,11.108250,-4.833150,5.385100,16.456800,0.393700,...,3.203600,7.347750,1.901300,3.396350,17.957950,-0.172700,2.408900,8.888200,15.934050,-2.819550
75%,0.000000,12.758200,1.358625,12.516700,8.324100,12.261125,0.924800,6.003000,19.102900,2.937900,...,6.406200,9.512525,2.949500,6.205800,20.396525,0.829600,6.556725,9.593300,18.064725,4.836800
max,1.000000,20.315000,10.376800,19.353000,13.188300,16.671400,17.251600,8.447700,27.691800,10.151300,...,18.440900,16.716500,8.402400,18.281800,27.928800,4.272900,18.321500,12.000400,26.079100,28.500700


In [252]:
train['target'].value_counts()

0    179902
1     20098
Name: target, dtype: int64

In [253]:
columns_variance=[train[col].var() for col in train.columns[2:]]

In [51]:
columns_variance

[9.241909296254263,
 16.40285794058822,
 6.974322132282455,
 4.175152602617283,
 2.634614409519367,
 61.83096293547501,
 0.7510081538599311,
 11.683240663326504,
 11.106446290054727,
 1.5253977203803928,
 30.25872370797137,
 35.64392202439157,
 0.03612251207720378,
 21.52528967438517,
 5.053091478198166,
 0.16950629489308655,
 6.540402138844952,
 45.059165737617775,
 61.64401466587767,
 63.947111864741515,
 34.530359963607,
 67.18365560330284,
 8.110864427020736,
 0.27761671125509313,
 14.267581524106065,
 0.08153049999252396,
 35.0725686923962,
 2.32170504162537,
 0.613664291894683,
 6.843149984916999,
 63.44438276774046,
 4.665127421719668,
 6.696861749472218,
 18.68249155139209,
 0.2933459379035042,
 26.827831424098203,
 9.734261692885541,
 5.061284521485547,
 18.309009471592272,
 16.555496918533027,
 68.5461327365231,
 35.26088426217378,
 0.48440360238838115,
 0.09585180318760592,
 34.846270320743535,
 458.1702382354569,
 8.182521178329633,
 111.93346997096565,
 129.60300968049432,

In [9]:
train.corr()['target'].sort_values()

var_81    -0.080917
var_139   -0.074080
var_12    -0.069489
var_146   -0.063644
var_76    -0.061917
var_174   -0.061669
var_21    -0.058483
var_166   -0.057773
var_80    -0.057609
var_165   -0.055734
var_13    -0.055156
var_148   -0.055011
var_198   -0.053000
var_34    -0.052692
var_115   -0.050174
var_109   -0.049926
var_44    -0.049039
var_169   -0.048382
var_149   -0.047319
var_92    -0.046295
var_154   -0.046106
var_108   -0.044791
var_33    -0.044334
var_192   -0.042858
var_9     -0.042805
var_122   -0.042461
var_123   -0.040291
var_107   -0.039997
var_121   -0.039788
var_86    -0.039126
             ...   
var_35     0.036567
var_155    0.037240
var_95     0.038531
var_89     0.039369
var_91     0.040127
var_147    0.040280
var_118    0.040358
var_164    0.040997
var_173    0.042022
var_18     0.043479
var_67     0.044673
var_94     0.046296
var_191    0.047114
var_170    0.047973
var_78     0.048245
var_184    0.048315
var_40     0.049530
var_179    0.050002
var_1      0.050343


In [254]:
X=train.drop(['target','ID_code'],axis=1,inplace=False)

In [255]:
X_test=test.drop('ID_code',axis=1,inplace=False)

In [256]:
y=train['target']

#### XGB Classifier 

In [90]:
X_train,X_validation,Y_train,Y_validation=train_test_split(X, y, test_size=0.3, random_state=42)

In [70]:
xgb_model= xgb.XGBClassifier( learning_rate =0.2, n_estimators=250, max_depth=8, min_child_weight=1, gamma=0, subsample=0.8,
 colsample_bytree=0.8, objective= 'binary:logistic', eval_metric="auc",nthread=4, scale_pos_weight=1, seed=27)

In [214]:
dtrain = xgb.DMatrix(X_train, Y_train)
dvalid = xgb.DMatrix(X_validation, Y_validation)

In [220]:
params = {
        "objective": "binary:logistic",
        "booster" : "gbtree",
        "eval_metric": "auc",
        "eta": 0.2,
        "tree_method": 'exact',
        "max_depth": 17,
        "subsample": 1,
        "colsample_bytree": 1,
        "silent": 1,
        "min_chil_weight":1,
        "seed": 42,
        #"num_class" : 22,
    }
num_boost_round = 2000
early_stopping_rounds = 50
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]

In [221]:
gbm = xgb.train(params, dtrain, num_boost_round, evals=watchlist, early_stopping_rounds=early_stopping_rounds, verbose_eval=True)


[0]	train-auc:0.794417	eval-auc:0.647241
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 50 rounds.
[1]	train-auc:0.860938	eval-auc:0.683979
[2]	train-auc:0.901269	eval-auc:0.704109
[3]	train-auc:0.929424	eval-auc:0.717268
[4]	train-auc:0.951667	eval-auc:0.725409
[5]	train-auc:0.963111	eval-auc:0.731121
[6]	train-auc:0.97229	eval-auc:0.736253
[7]	train-auc:0.983059	eval-auc:0.740522
[8]	train-auc:0.989305	eval-auc:0.746759
[9]	train-auc:0.993497	eval-auc:0.751596
[10]	train-auc:0.99663	eval-auc:0.755888
[11]	train-auc:0.997713	eval-auc:0.760159
[12]	train-auc:0.99897	eval-auc:0.763408
[13]	train-auc:0.999277	eval-auc:0.767835
[14]	train-auc:0.99957	eval-auc:0.771044
[15]	train-auc:0.999755	eval-auc:0.774304
[16]	train-auc:0.999931	eval-auc:0.777495
[17]	train-auc:0.999976	eval-auc:0.779998
[18]	train-auc:0.999986	eval-auc:0.783587
[19]	train-auc:0.999988	eval-auc:0.78677
[20]	train-auc:0.999998	eval-auc:0

[224]	train-auc:1	eval-auc:0.870947
[225]	train-auc:1	eval-auc:0.871004
[226]	train-auc:1	eval-auc:0.87107
[227]	train-auc:1	eval-auc:0.871185
[228]	train-auc:1	eval-auc:0.871194
[229]	train-auc:1	eval-auc:0.871252
[230]	train-auc:1	eval-auc:0.87129
[231]	train-auc:1	eval-auc:0.871352
[232]	train-auc:1	eval-auc:0.871436
[233]	train-auc:1	eval-auc:0.871485
[234]	train-auc:1	eval-auc:0.871608
[235]	train-auc:1	eval-auc:0.871674
[236]	train-auc:1	eval-auc:0.871734
[237]	train-auc:1	eval-auc:0.871851
[238]	train-auc:1	eval-auc:0.871956
[239]	train-auc:1	eval-auc:0.872035
[240]	train-auc:1	eval-auc:0.872126
[241]	train-auc:1	eval-auc:0.872171
[242]	train-auc:1	eval-auc:0.872199
[243]	train-auc:1	eval-auc:0.872258
[244]	train-auc:1	eval-auc:0.872348
[245]	train-auc:1	eval-auc:0.872415
[246]	train-auc:1	eval-auc:0.872466
[247]	train-auc:1	eval-auc:0.872489
[248]	train-auc:1	eval-auc:0.872538
[249]	train-auc:1	eval-auc:0.872624
[250]	train-auc:1	eval-auc:0.872639
[251]	train-auc:1	eval-auc:0.8

[453]	train-auc:1	eval-auc:0.878426
[454]	train-auc:1	eval-auc:0.878429
[455]	train-auc:1	eval-auc:0.87846
[456]	train-auc:1	eval-auc:0.878495
[457]	train-auc:1	eval-auc:0.878527
[458]	train-auc:1	eval-auc:0.87857
[459]	train-auc:1	eval-auc:0.878612
[460]	train-auc:1	eval-auc:0.878629
[461]	train-auc:1	eval-auc:0.87863
[462]	train-auc:1	eval-auc:0.878632
[463]	train-auc:1	eval-auc:0.878663
[464]	train-auc:1	eval-auc:0.878692
[465]	train-auc:1	eval-auc:0.878695
[466]	train-auc:1	eval-auc:0.878719
[467]	train-auc:1	eval-auc:0.878743
[468]	train-auc:1	eval-auc:0.878746
[469]	train-auc:1	eval-auc:0.878783
[470]	train-auc:1	eval-auc:0.878801
[471]	train-auc:1	eval-auc:0.878795
[472]	train-auc:1	eval-auc:0.878813
[473]	train-auc:1	eval-auc:0.878829
[474]	train-auc:1	eval-auc:0.878871
[475]	train-auc:1	eval-auc:0.878841
[476]	train-auc:1	eval-auc:0.878851
[477]	train-auc:1	eval-auc:0.878876
[478]	train-auc:1	eval-auc:0.878878
[479]	train-auc:1	eval-auc:0.878885
[480]	train-auc:1	eval-auc:0.87

[682]	train-auc:1	eval-auc:0.880953
[683]	train-auc:1	eval-auc:0.88097
[684]	train-auc:1	eval-auc:0.880979
[685]	train-auc:1	eval-auc:0.880993
[686]	train-auc:1	eval-auc:0.881014
[687]	train-auc:1	eval-auc:0.881025
[688]	train-auc:1	eval-auc:0.881047
[689]	train-auc:1	eval-auc:0.881054
[690]	train-auc:1	eval-auc:0.881062
[691]	train-auc:1	eval-auc:0.881074
[692]	train-auc:1	eval-auc:0.881061
[693]	train-auc:1	eval-auc:0.881059
[694]	train-auc:1	eval-auc:0.881066
[695]	train-auc:1	eval-auc:0.881058
[696]	train-auc:1	eval-auc:0.881089
[697]	train-auc:1	eval-auc:0.881089
[698]	train-auc:1	eval-auc:0.88109
[699]	train-auc:1	eval-auc:0.881105
[700]	train-auc:1	eval-auc:0.881105
[701]	train-auc:1	eval-auc:0.881102
[702]	train-auc:1	eval-auc:0.881123
[703]	train-auc:1	eval-auc:0.881136
[704]	train-auc:1	eval-auc:0.881164
[705]	train-auc:1	eval-auc:0.881163
[706]	train-auc:1	eval-auc:0.881186
[707]	train-auc:1	eval-auc:0.881188
[708]	train-auc:1	eval-auc:0.881206
[709]	train-auc:1	eval-auc:0.8

[910]	train-auc:1	eval-auc:0.882105
[911]	train-auc:1	eval-auc:0.882123
[912]	train-auc:1	eval-auc:0.882131
[913]	train-auc:1	eval-auc:0.882134
[914]	train-auc:1	eval-auc:0.882128
[915]	train-auc:1	eval-auc:0.882136
[916]	train-auc:1	eval-auc:0.882149
[917]	train-auc:1	eval-auc:0.882122
[918]	train-auc:1	eval-auc:0.882141
[919]	train-auc:1	eval-auc:0.882138
[920]	train-auc:1	eval-auc:0.882138
[921]	train-auc:1	eval-auc:0.882156
[922]	train-auc:1	eval-auc:0.882151
[923]	train-auc:1	eval-auc:0.88215
[924]	train-auc:1	eval-auc:0.882147
[925]	train-auc:1	eval-auc:0.882158
[926]	train-auc:1	eval-auc:0.882175
[927]	train-auc:1	eval-auc:0.88217
[928]	train-auc:1	eval-auc:0.882208
[929]	train-auc:1	eval-auc:0.882197
[930]	train-auc:1	eval-auc:0.88219
[931]	train-auc:1	eval-auc:0.882185
[932]	train-auc:1	eval-auc:0.882168
[933]	train-auc:1	eval-auc:0.882162
[934]	train-auc:1	eval-auc:0.882153
[935]	train-auc:1	eval-auc:0.88215
[936]	train-auc:1	eval-auc:0.882149
[937]	train-auc:1	eval-auc:0.882

[1135]	train-auc:1	eval-auc:0.882869
[1136]	train-auc:1	eval-auc:0.882878
[1137]	train-auc:1	eval-auc:0.882891
[1138]	train-auc:1	eval-auc:0.88288
[1139]	train-auc:1	eval-auc:0.882891
[1140]	train-auc:1	eval-auc:0.882909
[1141]	train-auc:1	eval-auc:0.882927
[1142]	train-auc:1	eval-auc:0.882929
[1143]	train-auc:1	eval-auc:0.882934
[1144]	train-auc:1	eval-auc:0.882937
[1145]	train-auc:1	eval-auc:0.882954
[1146]	train-auc:1	eval-auc:0.882961
[1147]	train-auc:1	eval-auc:0.882968
[1148]	train-auc:1	eval-auc:0.882984
[1149]	train-auc:1	eval-auc:0.882978
[1150]	train-auc:1	eval-auc:0.882976
[1151]	train-auc:1	eval-auc:0.882989
[1152]	train-auc:1	eval-auc:0.882986
[1153]	train-auc:1	eval-auc:0.88299
[1154]	train-auc:1	eval-auc:0.882981
[1155]	train-auc:1	eval-auc:0.882975
[1156]	train-auc:1	eval-auc:0.88297
[1157]	train-auc:1	eval-auc:0.882977
[1158]	train-auc:1	eval-auc:0.882986
[1159]	train-auc:1	eval-auc:0.882981
[1160]	train-auc:1	eval-auc:0.882993
[1161]	train-auc:1	eval-auc:0.883004
[116

[1358]	train-auc:1	eval-auc:0.88341
[1359]	train-auc:1	eval-auc:0.883412
[1360]	train-auc:1	eval-auc:0.883416
[1361]	train-auc:1	eval-auc:0.883426
[1362]	train-auc:1	eval-auc:0.883422
[1363]	train-auc:1	eval-auc:0.883417
[1364]	train-auc:1	eval-auc:0.883414
[1365]	train-auc:1	eval-auc:0.88341
[1366]	train-auc:1	eval-auc:0.883416
[1367]	train-auc:1	eval-auc:0.883406
[1368]	train-auc:1	eval-auc:0.883397
[1369]	train-auc:1	eval-auc:0.883398
[1370]	train-auc:1	eval-auc:0.883412
[1371]	train-auc:1	eval-auc:0.883411
[1372]	train-auc:1	eval-auc:0.88342
[1373]	train-auc:1	eval-auc:0.883411
[1374]	train-auc:1	eval-auc:0.883417
[1375]	train-auc:1	eval-auc:0.883434
[1376]	train-auc:1	eval-auc:0.883434
[1377]	train-auc:1	eval-auc:0.883432
[1378]	train-auc:1	eval-auc:0.883431
[1379]	train-auc:1	eval-auc:0.883435
[1380]	train-auc:1	eval-auc:0.883445
[1381]	train-auc:1	eval-auc:0.883454
[1382]	train-auc:1	eval-auc:0.883455
[1383]	train-auc:1	eval-auc:0.883461
[1384]	train-auc:1	eval-auc:0.883468
[138

In [222]:
y_pred = gbm.predict(xgb.DMatrix(X_test))


In [223]:
test['target'] = y_pred
#sub.to_csv('submission.csv', index=False)

In [224]:
test.loc[:,['ID_code','target']].to_csv('submission_lower_lr.csv', index=False)

In [ ]:
 fpr, tpr, _ = roc_curve(X_valid[target].values, check)
    roc_auc = auc(fpr, tpr)
    #xgb.plot_importance(gbm)
    #plt.show()
    plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',
             lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([-0.02, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve')
    plt.legend(loc="lower right")
    plt.show()

#### Stacking Models

In [257]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve, auc
from keras.models import Sequential
from keras.layers import Dense,Activation,BatchNormalization,Dropout
from keras.callbacks import ModelCheckpoint,EarlyStopping
import tensorflow as tf
from sklearn.utils import class_weight
from sklearn.preprocessing import MinMaxScaler

In [312]:
scaller=MinMaxScaler()
X=scaller.fit_transform(X)
X_test=scaller.fit_transform(X_test)

In [313]:
X

array([[0.42785307, 0.32482435, 0.56805853, ..., 0.43095798, 0.32765751,
        0.56064496],
       [0.55721218, 0.42863943, 0.6812351 , ..., 0.4682771 , 0.6095461 ,
        0.60582746],
       [0.41196889, 0.48377668, 0.57806091, ..., 0.38194973, 0.42583343,
        0.58273586],
       ...,
       [0.5432771 , 0.39305749, 0.4870996 , ..., 0.45612437, 0.38087342,
        0.63919915],
       [0.46750324, 0.25309006, 0.66682332, ..., 0.67445942, 0.46661746,
        0.37047369],
       [0.5258457 , 0.36714503, 0.580254  , ..., 0.36840624, 0.29469964,
        0.57890533]])

In [259]:
PartA_X, PartB_X, PartA_Y, PartB_Y = train_test_split(X, y, test_size=0.3, random_state=42)

In [227]:
# PartA_X, PartB_X, PartA_Y, PartB_Y = train_test_split(X, y, test_size=0.66, random_state=42)
# #split the test again to get 20% dev and 20% test
# PartB_X, PartC_X, PartB_Y, PartC_Y = train_test_split(PartB_X, PartB_Y, test_size=0.5, random_state=42)

#print(y_train.shape,y_dev.shape,y_test.shape)

In [228]:
PartA_X.shape,PartB_X.shape,PartC_X.shape

((68000, 200), (66000, 200), (66000, 200))

In [229]:
PartA_Y.shape,PartB_Y.shape,PartC_Y.shape

((68000,), (66000,), (66000,))

##### keras

In [338]:
model = Sequential()
model.add(Dense(128, input_dim=200))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dense(64))
model.add(Dropout(0.1))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dense(1))
model.add(Activation('sigmoid'))          
          
# model.add(Dense(1))
# model.add(Activation('sigmoid'))

In [339]:
class_weights = class_weight.compute_class_weight('balanced',np.unique(PartA_Y),PartA_Y)

In [340]:
np.unique(PartA_Y)

array([0, 1], dtype=int64)

In [341]:
class_weights

array([0.55535281, 5.01648273])

In [351]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['auc'])

ValueError: Unknown metric function:auc

In [346]:
filepath="model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=2, save_best_only=True, mode='max')


In [349]:
earlystopping=EarlyStopping(monitor='val_binary_accuracy', min_delta=0.001, patience=50, verbose=0, mode='max', baseline=None, restore_best_weights=True)


In [350]:
model.fit(PartA_X, PartA_Y,validation_data=(PartB_X, PartB_Y), epochs=1000, batch_size=100,verbose=2,
          callbacks=[checkpoint,earlystopping],class_weight=class_weights)

Train on 140000 samples, validate on 60000 samples
Epoch 1/1000
 - 4s - loss: 0.2331 - binary_accuracy: 0.9134 - val_loss: 0.2485 - val_binary_accuracy: 0.8984
Epoch 2/1000
 - 4s - loss: 0.2331 - binary_accuracy: 0.9138 - val_loss: 0.2531 - val_binary_accuracy: 0.8985
Epoch 3/1000
 - 4s - loss: 0.2331 - binary_accuracy: 0.9134 - val_loss: 0.2569 - val_binary_accuracy: 0.8982
Epoch 4/1000
 - 4s - loss: 0.2330 - binary_accuracy: 0.9135 - val_loss: 0.2513 - val_binary_accuracy: 0.8982
Epoch 5/1000
 - 4s - loss: 0.2332 - binary_accuracy: 0.9135 - val_loss: 0.2563 - val_binary_accuracy: 0.8984
Epoch 6/1000
 - 4s - loss: 0.2331 - binary_accuracy: 0.9137 - val_loss: 0.2538 - val_binary_accuracy: 0.8987
Epoch 7/1000
 - 4s - loss: 0.2335 - binary_accuracy: 0.9128 - val_loss: 0.2514 - val_binary_accuracy: 0.8984
Epoch 8/1000
 - 4s - loss: 0.2333 - binary_accuracy: 0.9135 - val_loss: 0.2543 - val_binary_accuracy: 0.8983
Epoch 9/1000
 - 4s - loss: 0.2333 - binary_accuracy: 0.9138 - val_loss: 0.253

In [321]:
y_pred = model.predict(X_test)
test['target'] = y_pred

In [322]:
np.sum(test['target'])

19850.758

In [323]:
test.loc[:,['ID_code','target']].to_csv('submission_lower_nn.csv', index=False)